# Introduction

## Node class

First we will start off with a simple node object, a node is a component of a graph and can have other nodes leading up to it, and nodes leading from it, but doesn't have to. Another optionality is to store data.

Here we introduce a simple node class, which just does this simplest function with methods get/set methods for the data, and the previous and next nodes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import binom


class Node:

    def __init__(self, previous_nodes=None, next_nodes=None, data=None):
        if previous_nodes is not None:
            self.previous_nodes = previous_nodes
        else:
            self.previous_nodes = []
        if next_nodes is not None:
            self.next_nodes = next_nodes
        else:
            self.next_nodes = []
        self.data = data

    def get_data(self):
        return self.data

    def set_data(self, data):
        self.data = data

    def get_previous_nodes(self):
        return self.previous_nodes

    def set_previous_nodes(self, nodes):
        self.previous_nodes.append(nodes)

    def get_next_nodes(self):
        return self.next_nodes

    def set_next_nodes(self, nodes):
        self.next_nodes.append(nodes)

Node objects present the basic building block of a tree which we intend to build, the code is pretty self-explanatory. One thing to note is that, technically the the next node of the current node, could be the node itself, this is a recurrent node, but for the purposes of this project, this will node be taken into consideration.

Next we present a binary tree, which is very similiar to the binomial tree which we will later on build, in the binomial tree pricing.


## Binary Tree

Binary tree, is usually recursively defined as a data structure where each parent node, has two child nodes, except for the last leaf nodes . Conventionally, nodes which do have subnodes in this data structure are called *interior nodes*, while the nodes in the last layer are called the *leaf nodes*.

Here we present the code for the binary tree

In [ ]:
class BinaryTree:
    def __init__(self, depth=10):
        self.tree = []
        self.depth = depth

    def create_tree(self):
        for i in range(1, self.depth + 1):
            self.tree.append([Node(data=i)] * (1 << i))
        len_all_nodes = ([len(x) for x in self.tree])
        for i in range(1, len(len_all_nodes)):
            counter = 0
            for j in range(len_all_nodes[i]):
                if j % 2 == 0 and j > 0:
                    counter = counter + 1
                if i > 0:
                    self.tree[i - 1][counter].set_next_nodes(self.tree[i][j])

Note, here bitshifting once is equivalent to stating $2^i$. While usually one would go about creating a tree differently, we are interested in easier traversal of the full tree structure, thus we opted not to set each of the interior nodes recursively by simply appending two nodes, then doing the same for the next node in the next layer. 

This would make things like computing the mean along a certain axis fairly more complex. Binary trees in their original form would be more suitable for things where there is a decision boundary that acts a sentinel for traversing the path.

You can access the data at each layer, by simply calling the nodes *get_data()* method, same goes for setting the data. By default the depth of the tree is set to 10, one could go for larger trees, in that case, the whole structure could be converted to numpy arrays in order to make the code work quicker, but in this project, we will not use a larger depth than 10.

## Binomial Tree

Binomial tree is a fairly similiar data structure to the binary tree, the key difference lies in it's the definitinion. Above we stated the definition for the binary tree, we should alse introduce the concept of left and right nodes, which are each of the subnodes of the inner nodes up to the leafs, furthermore, with distinct branches.

This simply means that, except for the leafs, each node has a left and right node, that are not connected. For the binomial tree, the recursive definition states that only the top-most and the bottom-most node have two distinct subnodes, while for all the other inner nodes, one of the nodes is distinct, while the other shares a connection with a previously defined node.

Below, we present the code to define a binary tree, same as before we did not create the tree recursively in order to make the data manipulation easier. Also each **Node** object is instantiated with dummy data, so the objects are not referencing the same object in memory. 

In [ ]:
class BinomialTree(BinaryTree):
    print_coo = []

    def __init__(self, depth=10):
        super().__init__(depth)

    def create_tree(self):
        self.tree.append([Node(data=0)])
        for s in range(1, self.depth):
            self.tree.append([])
            for j in range(s + 1):
                self.tree[s].append(Node(data=s))
        len_all_nodes = ([len(x) for x in self.tree])
        for k in range(len(len_all_nodes) - 1):
            for m in range(len_all_nodes[k]):
                self.tree[k][m].set_next_nodes(self.tree[k + 1][m])
                self.tree[k][m].set_next_nodes(self.tree[k + 1][m + 1])

    def set_tree_node(self, data, i, j):
        self.tree[i][j].set_data(data)

    def get_tree_node(self, i, j):
        return self.tree[i][j]